# Introduction

## __Project Goals__
Provide an overview of incidents that involve fireworks.
* Combine and Clean injury data


### About the Data

Source: [Firework Injury Reports](https://www.cpsc.gov/cgibin/NEISSQuery/UserCriteria.aspx?UserAff=5x08cgz9T6YPDAZJzvlZjA%3d%3d&UserAffOther=9OYR9kUytIsLilKZieD5xg%3d%3d)

The following notebook represents steps that were taken to organize and clean 5 excel files that consist of incident reports involving fireforks over the past 5 years from the CPSC (United States Consumer Product Safety Commision)


The data that was aquired was in excel format and was over 200MB in size. The data needed to be converted to csv since reading excel into pandas is extremely slow.  CSV does create a large file but the trade off for increased reading speeds is worth it.

### Tools Used: 
* Pandas

# Begin Cleaning Process

### Import Libraries

In [2]:
import pandas as pd

In [3]:
filelist = ['NEISS_2016.csv','NEISS_2017.csv','NEISS_2018.csv','NEISS_2019.csv','NEISS_2020.csv']
df = pd.concat(map(pd.read_csv, filelist))
df.head(3)

,CPSC_Case_Number,Treatment_Date,Age,Sex,Race,Other_Race,Hispanic,Body_Part,Diagnosis,Other_Diagnosis,...,Fire_Involvement,Alcohol,Drug,Product_1,Product_2,Product_3,Narrative,Stratum,PSU,Weight
0,160101845,1/1/16,92,1,0,NaN,NaN,79,57,NaN,...,0,NaN,NaN,1645,1807,0,92YOM TRYINGO TO TAKE OFF PANTS AND LOST BALAN...,M,63,103.2251
1,160101847,1/1/16,90,1,0,NaN,NaN,79,57,NaN,...,0,NaN,NaN,670,0,0,90YOM FELL GETTING OUT OF A RECLINER CHAIR AND...,M,63,103.2251
2,160101848,1/1/16,71,2,0,NaN,NaN,79,57,NaN,...,0,NaN,NaN,1807,0,0,71YOF SLIPPED AND FELL TO HER WET KITCHEN FLOO...,M,63,103.2251


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1791854 entries, 0 to 309369
Data columns (total 25 columns):
 #   Column             Dtype  
---  ------             -----  
 0   CPSC_Case_Number   int64  
 1   Treatment_Date     object 
 2   Age                int64  
 3   Sex                int64  
 4   Race               int64  
 5   Other_Race         object 
 6   Hispanic           float64
 7   Body_Part          int64  
 8   Diagnosis          int64  
 9   Other_Diagnosis    object 
 10  Body_Part_2        float64
 11  Diagnosis_2        float64
 12  Other_Diagnosis_2  object 
 13  Disposition        int64  
 14  Location           int64  
 15  Fire_Involvement   int64  
 16  Alcohol            float64
 17  Drug               float64
 18  Product_1          int64  
 19  Product_2          int64  
 20  Product_3          int64  
 21  Narrative          object 
 22  Stratum            object 
 23  PSU                int64  
 24  Weight             float64
dtypes: float64(6), int6

### Reduce DataFrame Size

Filter results to only diplay firework related incidents 
* *Fireworks product code is '1313'*

In [5]:
df1 = df.query('(Product_1 == 1313) | (Product_2 == 1313) | (Product_3 == 1313)')
df1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1532 entries, 57 to 308881
Data columns (total 25 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   CPSC_Case_Number   1532 non-null   int64  
 1   Treatment_Date     1532 non-null   object 
 2   Age                1532 non-null   int64  
 3   Sex                1532 non-null   int64  
 4   Race               1532 non-null   int64  
 5   Other_Race         88 non-null     object 
 6   Hispanic           701 non-null    float64
 7   Body_Part          1532 non-null   int64  
 8   Diagnosis          1532 non-null   int64  
 9   Other_Diagnosis    146 non-null    object 
 10  Body_Part_2        267 non-null    float64
 11  Diagnosis_2        267 non-null    float64
 12  Other_Diagnosis_2  28 non-null     object 
 13  Disposition        1532 non-null   int64  
 14  Location           1532 non-null   int64  
 15  Fire_Involvement   1532 non-null   int64  
 16  Alcohol            70

### Reduce Columns 

Remove unnecessary columns

In [6]:
# reduce df size and clean na values
df2 = df1[['Treatment_Date','Age','Sex','Body_Part','Diagnosis','Disposition','Location','Alcohol','Drug','Narrative']].fillna('').reset_index(drop=True)

df2.head()

,Treatment_Date,Age,Sex,Body_Part,Diagnosis,Disposition,Location,Alcohol,Drug,Narrative
0,1/1/16,39,1,77,53,1,1,,,39YOM WAS LIGHTING BOTTLE ROCKETS AND ONE FLEW...
1,1/1/16,10,1,82,51,1,1,,,10YOM SUSTAINED A THERMAL BURN TO HAND AFTER H...
2,1/1/16,35,1,31,62,4,0,,,35YOM HIT IN THE CHEST WITH A MORTAR TYPE FIRE...
3,1/1/16,13,1,77,53,1,0,,,13YOM SOMEONE POINTED FIREWORKS AT HIM FROM 10...
4,1/1/16,216,1,31,51,1,0,,,16MOM FAMILY PLAYING WITH FIREWORKS AND ONE SH...


In [7]:
# Fix Sex columns
df2.Sex = df2.Sex.replace(1,"Male").replace(2,"Female")
df2.head(2)

,Treatment_Date,Age,Sex,Body_Part,Diagnosis,Disposition,Location,Alcohol,Drug,Narrative
0,1/1/16,39,Male,77,53,1,1,,,39YOM WAS LIGHTING BOTTLE ROCKETS AND ONE FLEW...
1,1/1/16,10,Male,82,51,1,1,,,10YOM SUSTAINED A THERMAL BURN TO HAND AFTER H...


In [8]:
local_df = pd.read_pickle('df_incident_local.pkl')
local_df

,Code,Incident Locale
0,1,Home
1,2,Farm/Ranch
2,4,Street or highway
3,5,Other public property
4,6,Manufactured (mobile) home
5,7,Industrial place
6,8,School
7,9,Place of recreation or sports
8,0,Not recorded


In [9]:
# fix Incident Locale
df3 = pd.merge(df2, local_df, left_on='Disposition', right_on='Code').drop(['Code','Location'],axis=1)
df3.sample(3)

,Treatment_Date,Age,Sex,Body_Part,Diagnosis,Disposition,Alcohol,Drug,Narrative,Incident Locale
1347,12/31/19,35,Male,92,50,4,1.0,0.0,35 YOM WAS HOLDING A FIREWORK IN HIS HAND TOO ...,Street or highway
897,2/19/20,13,Male,77,53,1,0.0,0.0,13YOM PRESENTS AFTER LOOKING AT FIRECRACKER TY...,Home
1379,7/5/20,22,Female,75,62,4,0.0,0.0,22 YOF FELL HITTING THE BACK OF HIS HEAD ON CE...,Street or highway


In [10]:
body_part_df = pd.read_pickle('df_body_part.pkl')
body_part_df

,Code,Body_Part
0,0,Internal
1,30,Shoulder
2,31,Upper Trunk
3,32,Elbow
4,33,Lower Arm
5,34,Wrist
6,35,Knee
7,36,Lower Leg
8,37,Ankle
9,38,Pubic Region


In [11]:
# fix Body Part
df4 = pd.merge(df3, body_part_df, left_on='Body_Part', right_on='Code').drop(['Code','Body_Part_x'],axis=1)
df4.rename(columns={'Body_Part_y':'Body_Part'},inplace=True)
df4.sample(3)

,Treatment_Date,Age,Sex,Diagnosis,Disposition,Alcohol,Drug,Narrative,Incident Locale,Body_Part
546,7/3/20,21,Male,71,2,0.0,0.0,21YOM HAD A HOMEMADE FIREWORKS PIPE BOMB EXPLO...,Farm/Ranch,Hand
1012,7/1/18,35,Male,51,1,,,35YOM BURNED HIMSELF WITH A SPARKLER. DX: BURN...,Home,Finger
234,7/4/16,32,Male,51,1,,,32 YO M PT WAS TRYING TO LIGHT A HANDFUL OF SP...,Home,Hand


In [12]:
# import Diagnosis pkl
diagnosis_df = pd.read_pickle('df_diagnosis.pkl')
diagnosis_df

,Diagnosis1,Code
0,Ingested foreign object,41
1,Aspirated foreign object,42
2,"Burns, electrical 46 Burns, not specified",47
3,"Burns, scald (from hot liquids or steam)",48
4,"Burns, chemical (caustics, etc.)",49
5,Amputation,50
6,"Burns, thermal (from flames or hot surface)",51
7,Concussions,52
8,"Contusions, Abrasions",53
9,Crushing,54


In [13]:
# fix Diagnosis
df5 = pd.merge(df4, diagnosis_df, left_on='Diagnosis', right_on='Code').drop(['Code','Diagnosis'],axis=1)
df5.rename(columns={'Diagnosis1':'Diagnosis'},inplace=True)
df5.sample(3)

,Treatment_Date,Age,Sex,Disposition,Alcohol,Drug,Narrative,Incident Locale,Body_Part,Diagnosis
1377,7/5/20,64,Female,4,0.0,0.0,"64 YOF INJURED HIP WHILE LIGHTING FIREWORKS,TY...",Street or highway,Lower Trunk,Fracture
1,1/1/16,13,Male,1,,,13YOM SOMEONE POINTED FIREWORKS AT HIM FROM 10...,Home,Eyeball,"Contusions, Abrasions"
553,7/5/20,23,Male,1,0.0,0.0,23YOM PRESENTS AFTER BURN INJ PLAYING W/ FIREC...,Home,Hand,"Burns, thermal (from flames or hot surface)"


In [14]:
# import Disposition pkl
disposition_df = pd.read_pickle('df_disposition.pkl')
disposition_df

,Code,Disposition
0,1,Treated/Untreated and Released
1,2,Treated and transferred to another hospital
2,3,Treated and admitted for hospitalization
3,4,Held for observation
4,5,Left without being seen
5,6,Left against medical advice
6,7,Left without treatment
7,8,Eloped Fatality/DOA/died in the ED/Died after ...
8,9,Not recorded


In [15]:
# fix Disposition
df6 = pd.merge(df5, disposition_df, left_on='Disposition', right_on='Code').drop(['Code','Disposition_x'],axis=1)
df6.rename(columns={'Disposition_y':'Disposition'},inplace=True)
df6.sample(3)

,Treatment_Date,Age,Sex,Alcohol,Drug,Narrative,Incident Locale,Body_Part,Diagnosis,Disposition
673,7/8/17,39,Male,,,39YOM TO ED WITH BURN TO THUMB FROM SPARKLERS ...,Home,Finger,"Burns, thermal (from flames or hot surface)",Treated/Untreated and Released
1046,7/4/16,16,Male,,,16 YOM RUNNING FROM FRIEND PLAYING BOTTLE ROCK...,Home,Upper Trunk,Fracture,Treated/Untreated and Released
632,7/4/17,8,Male,,,8 YOM BURNED BY ROMAN CANDLE (TYPE I) SHOT @ H...,Home,Head,"Burns, thermal (from flames or hot surface)",Treated/Untreated and Released


In [17]:
# export out pickel of cleaned data
df6.to_pickle('clean.pkl')